# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

Prep Output Direction

In [3]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/Figures/regression_to_baseline/r_squared_plots'

Import Data

In [21]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/metadata/updated_with_all_adni/master_pivoted.csv'
sheet =  None #'master_list_proper_subjects'

In [22]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()


# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [24]:
drop_list = ['Visual', 'Measurement']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
data_df

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
data_df.columns

Set the parameters for dropping rows

In [43]:
column = 'Tissue'  # The column you'd like to evaluate
condition = 'not'  # The condition to check ('equal', 'above', 'below', 'not')
value = 'CSF' # The value to drop if T

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
data_df.columns

In [46]:
# Remove anything you don't want to standardize
# cols_not_to_standardize = ['subid'] #['Age']

In [ ]:
# data_df = cal_palm.standardize_columns(cols_not_to_standardize)
data_df

In [ ]:
data_df.columns

Regress out Covariate

In [49]:
# from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
# # use this code block to regress out covariates. Generally better to just include as covariates in a model..
# dependent_variable_list = lis
# regressors = ['Age', 'Sex']

# data_df, adjusted_dep_vars_list = RegressOutCovariates.run(df=data_df, dependent_variable_list=dependent_variable_list, covariates_list=regressors)
# print(adjusted_dep_vars_list)

# Get Proportions

In [ ]:
data_df.describe()

# Percent Values over a threshold

In [67]:
data_df.columns

Index(['Unnamed__0', 'subid', 'Age', 'Sex', 'Cohort', 'Diagnosis', 'Region',
       'Tissue', 'Visual', 'Measurement', 'Q4', 'TOTAL11', 'TOTALMOD'],
      dtype='object')

In [61]:
def calculate_percentage(df, column, threshold, filter_column, filter_value):
    """
    Calculate the percentage of rows in df where a specified column exceeds a threshold,
    given a filter condition on another column.

    Parameters:
    df (pd.DataFrame): The DataFrame to evaluate.
    column (str): The column to apply the threshold.
    threshold (float): The threshold value to exceed.
    filter_column (str): The column to filter by.
    filter_value (any): The value to filter the filter_column by.

    Returns:
    float: The percentage of rows meeting the conditions.
    """
    filtered_df = df[df[filter_column] == filter_value]
    count_above_threshold = filtered_df[filtered_df[column] > threshold].shape[0]
    percentage = (count_above_threshold / filtered_df.shape[0]) * 100 if filtered_df.shape[0] > 0 else 0
    return percentage


In [ ]:
calculate_percentage(df=data_df, column='Visual', threshold=0, filter_column='Cohort', filter_value=1)